# Group Analysis - FOOOFed EEG Analysis: Task
    
Applying FOOOF to task based EEG data, and comparing between young and old groups.

Notes:
- average power spectra vs. average FOOOFs
    - there are 10 segments that don't return a value in the single channel version
        - segments, with all-channel or canonical, have systematically different (lower) alpha power
    - best performance (for both canonical & FOOOF) is from dropping those segments
    - when doing single channel FOOOF, there is a benefit of other alpha features (CF & BW) 

Status:
- FOOOF fits can help select data to analyze
- Gives more features to use to predict behaviour, and can outperform canonical when these features are combined

In [62]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import warnings
from os.path import join as pjoin
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import pearsonr, spearmanr

import patsy
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.diagnostic import compare_cox, compare_j

from fooof.synth.gen import gen_aperiodic

In [64]:
# Import custom code for this analysis
%autoreload 2
from fio import *
from plts import *
from utils import *
from data_mgmt import *
from analysis import *

# Settings

In [65]:
# Set path to load results from
res_path = '/Users/tom/Documents/Research/1-Projects/fooof/2-Data/Results/'

# Set indices to separate groups
from settings import YNG_INDS, OLD_INDS

# Set which group of FOOOF results to load
folder = 'FOOOF-SinCh'        # FOOOF fit on an average across power spectra 
#folder = 'FOOOF-AllCh'       # Average across FOOOF fits, from each channel

# Wether to save out plots or not
save_fig = False

print('Number of young subjects:  ', len(YNG_INDS))
print('Number of  old  subjects:  ', len(OLD_INDS))

Number of young subjects:   17
Number of  old  subjects:   14


In [66]:
# Data settings
srate = 512
tmin, tmax = -0.85, 1.1
times = np.arange(tmin, tmax, 1/srate)
seg_times = [(-0.85, -0.35), (0.1, 0.6), (0.5, 1.0)]
n_subjs = 31

### Check dropped trials

In [67]:
# Load dropped trials & components
dropped_trials = np.load(pjoin(res_path, 'Group', 'dropped_trials.npy'))
dropped_components = np.load(pjoin(res_path, 'Group', 'dropped_components.npy'))

# Check dropped trials for each subject
print('SubNum \t\t # Dropped Trials \t # Dropped Components')
for ind, trials, components in zip(range(n_subjs), dropped_trials, dropped_components):
    temp_trials = trials[trials < 999.]
    temp_comps = components[components < 999.]
#    print(ind, '\t\t', len(temp_trials), '\t\t\t', len(temp_comps))

SubNum 		 # Dropped Trials 	 # Dropped Components


## Group FOOOFing - Trial Averaged Data

Notes:
- 3D data objects have the shape `[n_loads, n_subjs, n_times]`

### Load Data

In [68]:
# Load behavioural data
behav_dat = pd.read_csv(pjoin(res_path, 'Behav', 'neural_aging_data_behaviour.csv'))

In [69]:
# Convert data types
behav_dat['Age'] = behav_dat['Age'].astype('str')
behav_dat['Load'] = behav_dat['Load'].astype('str')

In [70]:
# Calculate average behaviour across loads
avg_behav = behav_dat.groupby('SubjID').mean()

In [71]:
# Select age group of subjects
#behav_dat = behav_dat[behav_dat['Age'] == '2']

In [72]:
# CHEATING
#for ind in [28, 59, 90]:
#    behav_dat.loc[ind, "d'"] = 2

In [73]:
# Load and extract FOOOF data
all_offsets, all_exps = load_fooof_task_ap(res_path, 'Contra', folder)
all_alphas_cf = load_fooof_task_pe(res_path, 'Contra', 0, folder)
all_alphas =  load_fooof_task_pe(res_path, 'Contra', 1, folder)
all_alphas_bw = load_fooof_task_pe(res_path, 'Contra', 2, folder)

In [74]:
# Replace NaN with row mean
# import numpy.ma as ma
# out = []
# for mat in all_alphas:
#     mat = mat.T
#     out.append(np.where(np.isnan(mat), ma.array(mat, mask=np.isnan(mat)).mean(axis=0), mat).T)
# all_alphas = np.stack(out)

# Replace nan with global mean
#all_alphas[np.isnan(all_alphas)] = np.nanmean(all_alphas)

In [75]:
# Load canonical alpha analysis
canonical_group = np.load(pjoin(res_path, 'Group', 'canonical_group.npy'))

In [76]:
# Average across analytic alpha measures to get canonical alpha measure
seg_masks = []
for seg in seg_times:
    seg_masks.append(np.logical_and(times >= seg[0], times <= seg[1]))

canalpha = np.zeros_like(all_alphas)
for subi, subj_dat in enumerate(canonical_group):
    for lodi in range(3):
        for segi, mask in enumerate(seg_masks):
            canalpha[lodi, subi, segi] = np.mean(subj_dat[lodi, mask])

In [77]:
# # Load and extract FOOOF data
# all_offsets = all_offsets[:, YNG_INDS, :]
# all_exps = all_exps[:, YNG_INDS, :]
# all_alphas_cf = all_alphas_cf[:, YNG_INDS, :]
# all_alphas = all_alphas[:, YNG_INDS, :]
# all_alphas_bw = all_alphas_bw[:, YNG_INDS, :]
# canalpha = canalpha[:, YNG_INDS, :]

In [78]:
# Number of missing FOOOFed alphas
sum(sum(sum(np.isnan(all_alphas))))

10

In [79]:
# Check where the NaN values are
#nans = np.isnan(all_alphas)

In [80]:
# # Drop missing FOOOF values from other measures
# canalpha[nans] = np.nan
# all_exps[nans] = np.nan
# all_offsets[nans] = np.nan

In [81]:
# # TEST
# all_alphas[nans] = np.nan
# all_alphas_cf[nans] = np.nan
# all_alphas_bw[nans] = np.nan

In [82]:
# # TESTS - drop particular subjects
# for drop_ind in [1, 8]:
#     all_alphas[:, 1, :] = np.nan
#     canalpha[:, 1, :] = np.nan
#     all_exps[:, 1, :] = np.nan
#     all_offsets[:, 1, :] = np.nan

In [83]:
# CHEATING: DROP OUT SUBJECTS
#behav_dat[behav_dat['SubjID'] == 8] = np.nan
#behav_dat[behav_dat['SubjID'] == 1] = np.nan

In [84]:
nan_inds = np.where(np.isnan(all_alphas))

In [85]:
sum(behav_dat['SubjID'] == 1)

3

In [86]:
# `[n_loads, n_subjs, n_times]`

# df[(df>=0)&(df<=20)].dropna()

In [87]:
#set(behav_dat.SubjID.values)

In [88]:
# # Drop behavioural data where the EEG data is NaN
# all_inds = YNG_INDS + OLD_INDS
# for load_ind, subj_ind, times_ind in zip(*nan_inds):
#     if times_ind == 0 or times_ind == 2:
#         index = behav_dat[(behav_dat['Load'] == str(load_ind + 1)) & \
#                           (behav_dat['SubjID'] == subj_ind + 1)].index
#         print(load_ind+1, subj_ind, times_ind, '\t', index)
#         behav_dat.loc[index, "d'"] = np.nan

In [89]:
qq = np.where(np.isnan(np.concatenate([al_dif_yng, al_dif_old])))
qq[0]

NameError: name 'al_dif_yng' is not defined

In [90]:
behav_dat.query?

In [91]:
ww = np.vstack([al_st_yng, al_st_old])
ww.shape

NameError: name 'al_st_yng' is not defined

In [92]:
np.where(np.isnan(ww))

NameError: name 'ww' is not defined

In [93]:
behav_dat.iloc[8]

SubjID         9
Age            1
Load           1
CDA      -2.1048
d'        4.2064
Name: 8, dtype: object

In [94]:
behav_dat.iloc[74]

SubjID         13
Age             1
Load            3
CDA      -2.29152
d'        3.93454
Name: 74, dtype: object

In [95]:
np.where(np.isnan(al_dif_old))

NameError: name 'al_dif_old' is not defined

In [96]:
al_dif_old.shape

NameError: name 'al_dif_old' is not defined

In [97]:
nan_inds

(array([0, 0, 1, 1, 1, 2, 2, 2, 2, 2]),
 array([ 1,  8,  1,  1,  8,  1,  1,  3,  8, 29]),
 array([1, 0, 1, 2, 0, 1, 2, 1, 2, 2]))

In [98]:
behav_dat[np.isnan(behav_dat["d'"])]

,SubjID,Age,Load,CDA,d'


In [99]:
sum(np.isnan(al_dif_old)) + sum(np.isnan(al_dif_yng))

NameError: name 'al_dif_old' is not defined

In [100]:
sum(np.isnan(behav_dat["d'"].values))

0

In [101]:
#all_inds

In [102]:
nans = np.isnan(all_alphas)

In [103]:
print(np.mean(all_alphas[nans]))
print(np.mean(all_alphas[~nans]))

nan
0.5009890129868549


In [104]:
print(np.mean(all_alphas_cf[nans]))
print(np.mean(all_alphas_cf[~nans]))

nan
10.718838504875412


In [105]:
print(np.mean(all_alphas_bw[nans]))
print(np.mean(all_alphas_bw[~nans]))

nan
3.5223403743838793


In [106]:
print(np.mean(canalpha[nans]))
print(np.mean(canalpha[~nans]))

-3.3513236216584837e-07
-8.196500058385066e-08


## Data Management

In [107]:
# Set up th
labels = ['offset', 'exponent', 'alpha_cf', 'alpha_pw', 'alpha_bw', 'canalpha']
datas = [all_offsets, all_exps, all_alphas_cf, all_alphas, all_alphas_bw, canalpha]

# Make a data dictionary - each with shape [n_conds, n_times]
data_dict = {'YNG' : {}, 'OLD' : {}, 'ALL' : {}}
diff_data_dict = deepcopy(data_dict)
behav_dict = deepcopy(data_dict)
i1, i2 = 2, 0

In [108]:
# Set up data & diff_data dicts
for label, data in zip(labels, datas):
    data_dict['YNG'][label], data_dict['OLD'][label] = reshape_dat(data)
    data_dict['ALL'][label] = np.concatenate([data_dict['YNG'][label], data_dict['OLD'][label]])
    
    diff_data_dict['YNG'][label] = calc_diff(data_dict['YNG'][label], i1, i2)
    diff_data_dict['OLD'][label] = calc_diff(data_dict['OLD'][label], i1, i2)
    diff_data_dict['ALL'][label] = np.concatenate([diff_data_dict['YNG'][label], diff_data_dict['OLD'][label]])

In [109]:
# Set up the behavioural data dict
for label in ["d'", "Load", 'CDA']:
    behav_dict['ALL'][label] = behav_dat[label].values
    behav_dict['YNG'][label] = behav_dat[behav_dat['Age'] == '1'][label].values
    behav_dict['OLD'][label] = behav_dat[behav_dat['Age'] == '2'][label].values

In [110]:
# ## ReOrg data - reshape 3D matrix, to 2D matrix with stacked trials
# canal_st_yng, canal_st_old = reshape_dat(canalpha)

# off_st_yng, off_st_old = reshape_dat(all_offsets)
# exp_st_yng, exp_st_old = reshape_dat(all_exps)

# al_st_yng, al_st_old = reshape_dat(all_alphas)
# al_cf_st_yng, al_cf_st_old = reshape_dat(all_alphas_cf)
# al_bw_st_yng, al_bw_st_old = reshape_dat(all_alphas_bw)

In [111]:
# # Calculate difference measures
# i1, i2 = 2, 0

# canal_dif_yng = calc_diff(canal_st_yng, i1, i2)
# canal_dif_old = calc_diff(canal_st_old, i1, i2)

# off_dif_yng = calc_diff(off_st_yng, i1, i2)
# off_dif_old = calc_diff(off_st_old, i1, i2)

# exp_dif_yng = calc_diff(exp_st_yng, i1, i2)
# exp_dif_old = calc_diff(exp_st_old, i1, i2)

# al_dif_yng = calc_diff(al_st_yng, i1, i2)
# al_dif_old = calc_diff(al_st_old, i1, i2)

# al_cf_dif_yng = calc_diff(al_cf_st_yng, i1, i2)
# al_cf_dif_old = calc_diff(al_cf_st_old, i1, i2)

# al_bw_dif_yng = calc_diff(al_bw_st_yng, i1, i2)
# al_bw_dif_old = calc_diff(al_bw_st_old, i1, i2)

In [112]:
# print('CANALPHA - YNG: ', np.nanmean(canal_dif_yng))
# print('CANALPHA - OLD: ', np.nanmean(canal_dif_old))

# print('')
# print('Alpha Power - YNG: ', np.nanmean(al_dif_yng))
# print('Alpha Power - OLD: ', np.nanmean(al_dif_old))
# print('')
# print('Alpha CF - YNG: ', np.nanmean(al_cf_dif_yng))
# print('Alpha CF - OLD: ', np.nanmean(al_cf_dif_old))
# print('')
# print('Alpha BW - YNG: ', np.nanmean(al_bw_dif_yng))
# print('Alpha BW - OLD: ', np.nanmean(al_bw_dif_old))

# print('')
# print('Exp - YNG: ', np.nanmean(exp_dif_yng))
# print('Exp - OLD: ', np.nanmean(exp_dif_old))
# print('')
# print('Off - YNG: ', np.nanmean(exp_dif_yng))
# print('Off - OLD: ', np.nanmean(exp_dif_old))

In [113]:
# Print out mean values per group
print('\t\t   YNG \t\t  OLD')
for label in labels:
    print_stat(label,
               np.nanmean(diff_data_dict['YNG'][label]),
               np.nanmean(diff_data_dict['OLD'][label]))

		   YNG 		  OLD
offset: 	 -0.0717 	-0.0888
exponent: 	 -0.0484 	-0.0428
alpha_cf: 	 -0.1245 	 0.6765
alpha_pw: 	  0.0816 	-0.0051
alpha_bw: 	 -0.2969 	 0.2854
canalpha: 	  0.0000 	-0.0000


In [114]:
# Print out tests for group differences
print('\t\t   YNG \t\t  OLD')
for label in labels:
    print_stat(label, *nan_ttest(diff_data_dict['YNG'][label], diff_data_dict['OLD'][label]))

		   YNG 		  OLD
offset: 	  0.6957 	 0.4884
exponent: 	 -0.2416 	 0.8096
alpha_cf: 	 -3.8230 	 0.0003
alpha_pw: 	  2.7494 	 0.0073
alpha_bw: 	 -3.7894 	 0.0003
canalpha: 	  2.2754 	 0.0252


In [115]:
# print_stat('CanAl', *nan_ttest(canal_dif_yng, canal_dif_old))

# print('')
# print_stat('Al-CF', *nan_ttest(al_cf_dif_yng, al_cf_dif_old))
# print_stat('Al-PW', *nan_ttest(al_dif_yng, al_dif_old))
# print_stat('Al-BW', *nan_ttest(al_bw_dif_yng, al_bw_dif_old))

# print('')
# print_stat('Exp', *nan_ttest(exp_dif_yng, exp_dif_old))
# print_stat('Off', *nan_ttest(off_dif_yng, off_dif_old))

In [116]:
# Print out tests for group differences
print('\t\t   YNG \t\t  OLD')
for label in labels:
    print_stat(label,
    nan_corr(diff_data_dict['YNG'][label], behav_dict['YNG']["d'"])[0],
    nan_corr(diff_data_dict['OLD'][label], behav_dict['OLD']["d'"])[0])

		   YNG 		  OLD
offset: 	  0.1978 	 0.0474
exponent: 	  0.3171 	-0.1065
alpha_cf: 	  0.1978 	 0.0086
alpha_pw: 	 -0.1362 	 0.3408
alpha_bw: 	  0.1042 	 0.0289
canalpha: 	 -0.1787 	 0.2544


In [117]:
# # Check correlations of difference measures with behaviour
# print_stat('Sl-Yng', *nan_corr(exp_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
# print_stat('Sl-Old', *nan_corr(exp_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))
# print('')
# print_stat('CanAl-Yng', *nan_corr(canal_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
# print_stat('CanAl-Old', *nan_corr(canal_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))
# print('')
# print_stat('Al-Yng', *nan_corr(al_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
# print_stat('Al-Old', *nan_corr(al_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))
# print('')
# print_stat('Al-CF-Yng', *nan_corr(al_cf_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
# print_stat('Al-CF-Old', *nan_corr(al_cf_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))
# print('')
# print_stat('Al-BW-Yng', *nan_corr(al_bw_dif_yng, behav_dat[behav_dat.Age == '1']["d'"]))
# print_stat('Al-BW-Old', *nan_corr(al_bw_dif_old, behav_dat[behav_dat.Age == '2']["d'"]))

# Difference Measures

Predict behaviour output from evoked responses of alpha and aperiodic. 

In [118]:
# MODEL SETTINGS
group = 'ALL' # 'ALL', 'YNG', 'OLD'

### Base Model (Predict from load)

In [119]:
model_def = 'behav ~ load'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load']}
base_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     6.736
Date:                Wed, 06 Mar 2019   Prob (F-statistic):            0.00188
Time:                        16:56:33   Log-Likelihood:                -123.85
No. Observations:                  93   AIC:                             253.7
Df Residuals:                      90   BIC:                             261.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8056      0.167     22.743      0.0

### CDA Model

In [120]:
model_def = 'behav ~ load + cda'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load'],
            'cda' : behav_dict[group]['CDA']}
cda_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     4.442
Date:                Wed, 06 Mar 2019   Prob (F-statistic):            0.00589
Time:                        16:56:33   Log-Likelihood:                -123.85
No. Observations:                  93   AIC:                             255.7
Df Residuals:                      89   BIC:                             265.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8009      0.189     20.129      0.0

### Canonical Alpha Model

In [121]:
model_def = 'behav ~ load + al_dif'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load'],
            'al_dif' : diff_data_dict['ALL']['canalpha']}
canalpha_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     4.517
Date:                Wed, 06 Mar 2019   Prob (F-statistic):            0.00538
Time:                        16:56:34   Log-Likelihood:                -123.75
No. Observations:                  93   AIC:                             255.5
Df Residuals:                      89   BIC:                             265.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7960      0.169     22.398      0.0

### FOOOF Alpha Model

In [122]:
model_def = 'behav ~ load + al_dif'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load'],
            'al_dif' : diff_data_dict[group]['alpha_pw']}
f_alpha_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     5.228
Date:                Wed, 06 Mar 2019   Prob (F-statistic):            0.00235
Time:                        16:56:34   Log-Likelihood:                -115.59
No. Observations:                  87   AIC:                             239.2
Df Residuals:                      83   BIC:                             249.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7603      0.178     21.090      0.0

### FOOOF Alpha+ Model

In [123]:
model_def = 'behav ~ load + al_cf_dif + al_dif + al_bw_dif'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load'],
            'al_cf_dif' : diff_data_dict[group]['alpha_cf'],
            'al_dif' : diff_data_dict[group]['alpha_pw'],
            'al_bw_dif' : diff_data_dict[group]['alpha_bw']}
f_alpha_plus_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     4.400
Date:                Wed, 06 Mar 2019   Prob (F-statistic):            0.00136
Time:                        16:56:35   Log-Likelihood:                -112.67
No. Observations:                  87   AIC:                             237.3
Df Residuals:                      81   BIC:                             252.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.6444      0.181     20.103      0.0

## Predicting from FOOOF: Aperiodic

In [124]:
model_def = 'behav ~ load + exp_dif + off_dif'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load'],
            'off_dif' : diff_data_dict[group]['offset'],
            'exp_dif' : diff_data_dict[group]['exponent']}
f_alpha_plus_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     3.611
Date:                Wed, 06 Mar 2019   Prob (F-statistic):            0.00898
Time:                        16:56:35   Log-Likelihood:                -123.27
No. Observations:                  93   AIC:                             256.5
Df Residuals:                      88   BIC:                             269.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8603      0.183     21.133      0.0

### Predicting from FOOOF model with Alpha & Aperiodic

In [125]:
model_def = 'behav ~ load + al_dif + exp_dif + off_dif'
data_def = {'behav' : behav_dict[group]["d'"],
            'load' : behav_dict[group]['Load'],
            'al_dif' : diff_data_dict[group]['alpha_pw'],
            'off_dif' : diff_data_dict[group]['offset'],
            'exp_dif' : diff_data_dict[group]['exponent']}
f_alpha_plus_model = run_model(model_def, data_def)

                            OLS Regression Results                            
Dep. Variable:                  behav   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     3.170
Date:                Wed, 06 Mar 2019   Prob (F-statistic):             0.0115
Time:                        16:56:36   Log-Likelihood:                -115.35
No. Observations:                  87   AIC:                             242.7
Df Residuals:                      81   BIC:                             257.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7951      0.201     18.906      0.0

## SOMETHING ELSE

In [126]:
# loadi = 2
# time_exps = all_exps[loadi, inds, :].mean(0)
# time_offs = all_offsets[loadi, inds, :].mean(0)

# time_al_cf = all_alphas_cf[loadi, inds, :].mean(0)
# time_al_pw = all_alphas[loadi, inds, :].mean(0)
# time_al_bw = all_alphas_bw[loadi, inds, :].mean(0)

In [ ]:
np.median(al_dif_old)

In [ ]:
inds = YNG_INDS

In [ ]:
time_exps = all_exps[:, inds, :].mean(0).mean(0)
time_offs = all_offsets[:, inds, :].mean(0).mean(0)

time_al_cf = all_alphas_cf[:, inds, :].mean(0).mean(0)
time_al_pw = all_alphas[:, inds, :].mean(0).mean(0)
time_al_bw = all_alphas_bw[:, inds, :].mean(0).mean(0)

In [ ]:
print(time_al_pw)

In [ ]:
fs, base_spectrum = gen_power_spectrum([3, 30],
                                       [time_offs[0], time_exps[0]],
                                       [time_al_cf[0], time_al_pw[0], time_al_bw[0]/2], nlv=0)
fs, task_spectrum = gen_power_spectrum([3, 30],
                                       [time_offs[2], time_exps[2]],
                                       [time_al_cf[2], time_al_pw[2], time_al_bw[2]/2], nlv=0)

In [ ]:
_, ax = plt.subplots(figsize=(6, 6))
plot_spectra(fs, [base_spectrum, task_spectrum], True, True, ax=ax)

In [ ]:
from fooof.synth.gen import gen_power_spectrum
from fooof.plts import plot_spectra

In [ ]:
gen_power_spectrum?

In [ ]:
np.mean(al_dif_old)

## Model Comparisons

Explicitly test for differences between different model fits. 

#### Comparing Nested Models

Statsmodels offers three tests for nested models: f test, lagrange multiplier, likelihood ratio

Note that these three can be called from a results object, as `compare_x_test` with `f`, `lm` and `lr` as `x`. 

F-test can also be run with `anova_lm`. 

#### Comparing Non-Nested Models

Statmodels offers two tests for non-nested model: cox test & j test

They are better described in the R implementations:

- cox_test: http://math.furman.edu/~dcs/courses/math47/R/library/lmtest/html/coxtest.html
- j_test: http://math.furman.edu/~dcs/courses/math47/R/library/lmtest/html/jtest.html

In [ ]:
# Compare nested models: alpha models vs base models
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('Canonical alpha vs. Base Model')
    print(anova_lm(mod_base, mod_alpha_can))
    print('\n')
    print('FOOOFed alpha vs. Base Model')
    print(anova_lm(mod_base, mod_alpha_foo))

In [ ]:
# Compare different alpha models
print('Canonical alpha vs. FOOOFed Alpha')
print_stat('Alpha-Model Compare', *compare_cox.run(mod_alpha_can, mod_alpha_foo))

In [ ]:
# Compare different alpha models
print('Canonical alpha vs. FOOOFed Alpha')
print_stat('Alpha-Model Compare', *compare_cox.run(mod_alpha_can, mod_alpha_foo_all))

In [ ]:
# Compare nested model: aperiodic exponent
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('FOOOF aperiodic vs. Base Model')
    print(anova_lm(mod_base, mod_exponent_foo))    

In [ ]:
# Compare the full FOOOF model to the one with canonical alpha
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print('FOOOF Full vs. Canonical Alpha')
    print(anova_lm(mod_alpha_can, mod_all_foo))

In [ ]:
# Compare FOOOF model with exponent to restricted model with just alpha (from FOOOF)
lm_val, p_val, df_diff = mod_all_foo.compare_f_test(mod_alpha_foo)
print_stat('Model comp', lm_val, p_val)

## Correlations

Check the correlational structure between canonical and FOOOF features. 

In [ ]:
# Generate the power @ alpha frequency given the aperiodic component, from the FOOOF fits
ap_alpha = []
for cf, off, exp in zip(all_alphas_cf.flatten(), all_offsets.flatten(), all_exps.flatten()):
    ap_alpha.append(gen_aperiodic(np.array([10]), [off, exp])[0])
ap_alpha = np.array(ap_alpha)

# Calculate the total power at 10 Hz (or about) from the 
foo_total = ap_alpha + all_alphas.flatten()

# Find NaN locations
nan_inds = np.isnan(ap_alpha)

In [ ]:
# Calculate correlation between canonical and FOOOF alpha
print_stat('CanalvsFAlpha', *pearsonr(np.array(canalpha.flatten())[~nan_inds], np.array(all_alphas.flatten())[~nan_inds]))

In [ ]:
# Calculate correlation between canonical alpha and aperiodic component @ 10 Hz
print_stat('Canalvs10HzAP', *pearsonr(np.array(canalpha.flatten())[~nan_inds], np.array(ap_alpha)[~nan_inds]))

In [ ]:
# Calculate correlation between the canonical alpha and the FOOOF model total @ 10 Hz
print_stat('CanalvsTotalFOOOF',*pearsonr(np.array(canalpha.flatten())[~nan_inds], np.array(foo_total.flatten())[~nan_inds]))

In [ ]:
# Check correlation between exponent & d', and alpha & d' - for a specific load and time point

# Set which load and time point to use
l_ind = 0
t_ind = 0

print_stat("Sl-Lo-d'", *pearsonr(all_exps[l_ind, :, t_ind],
                                 behav_dat[behav_dat.Load == str(l_ind + 1)]["d'"]))
print_stat("Al-Lo-d'", *pearsonr(all_alphas[l_ind, :, t_ind],
                                 behav_dat[behav_dat.Load == str(l_ind + 1)]["d'"]))

In [ ]:
t_ind = 2

print_stat("Sl-d'", *pearsonr(all_exps[:, :, t_ind].flatten(), behav_dat["d'"].values))
print_stat("Al-d'", *pearsonr(all_alphas[:, :, t_ind].flatten(), behav_dat["d'"].values))

In [ ]:
# Average d' with average measures
t_ind = 2
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_exps[:, :, t_ind], 0)))
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_alphas[:, :, t_ind], 0)))

In [ ]:
# Average d' with average measures: diff measures
t_ind = 2
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_exps[:, :, 2] - all_exps[:, :, 0], 0)))
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_alphas[:, :, 2] - all_alphas[:, :, 0], 0)))

In [ ]:
ind = 2

print_stat('Yng-Sl', *pearsonr(exp_st_yng[:, ind], behav_dat[behav_dat.Age == '1']["d'"]))
print_stat('Old-Sl', *pearsonr(exp_st_old[:, ind], behav_dat[behav_dat.Age == '2']["d'"]))

print('')

print_stat('Yng-Al', *pearsonr(al_st_yng[:, ind], behav_dat[behav_dat.Age == '1']["d'"]))
print_stat('Old-Al', *pearsonr(al_st_old[:, ind], behav_dat[behav_dat.Age == '2']["d'"]))

print('')

print_stat('Yng-CanAl', *pearsonr(canal_st_yng[:, ind], behav_dat[behav_dat.Age == '1']["d'"]))
print_stat('Old-CanAl', *pearsonr(canal_st_old[:, ind], behav_dat[behav_dat.Age == '2']["d'"]))

In [ ]:
# Average across loads
sl_resps = np.diff(np.mean(all_exps, 0))
al_resps = np.diff(np.mean(all_alphas, 0))

# # Take specific load
# load_ind = 2
# sl_resps = all_exps[load_ind, :, :]
# al_resps = all_alphas[load_ind, :, :]

In [ ]:
# Correlations between reactivity measures
print_stat('SLR & ALR - 1', *pearsonr(sl_resps[:, 0], al_resps[:, 0]))
print_stat('SLR & ALR - 2', *pearsonr(sl_resps[:, 1], al_resps[:, 1]))

In [ ]:
# ind = 0
# edat = sl_resps

# print_stat('\tALL', *pearsonr(avg_behav["d'"], al_resps[:, 0]))
# print_stat('\tYNG', *pearsonr(avg_behav[avg_behav['Age'] == '1']["d'"], edat[YNG_INDS, ind]))
# print_stat('\tOLD', *pearsonr(avg_behav[avg_behav['Age'] == '2']["d'"], edat[OLD_INDS, ind]))

In [ ]:
# edat = sl_resps

# print_stat('\tALL', *pearsonr(avg_behav["d'"], al_resps[:, 0]))
# print_stat('\tYNG', *pearsonr(avg_behav[avg_behav['Age'] == 1]["d'"], np.mean(edat[YNG_INDS], 1)))
# print_stat('\tOLD', *pearsonr(avg_behav[avg_behav['Age'] == 2]["d'"], np.mean(edat[OLD_INDS], 1)))

In [ ]:
print(pearsonr(avg_behav["CDA"], al_resps[:, 0]))
print(pearsonr(avg_behav["CDA"], sl_resps[:, 0]))

In [ ]:
print(pearsonr(exp_dif_yng, al_dif_yng))
print(pearsonr(exp_dif_old, al_dif_old))

## Miscellaneous Crap

In [ ]:
# # Build the dataframe

# t_ind = 2
# l_ind = 0 

# df = pd.DataFrame()

# df['behav'] = behav_dat[behav_dat['Load'] == l_ind+1]["d'"].values
# df['age'] = behav_dat[behav_dat['Load'] == l_ind+1]["Age"].values

# df['exp_dif'] = np.concatenate([exp_dif_yng, exp_dif_old])
# df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])

# # 
# outcome, predictors = patsy.dmatrices("behav ~ exp_dif + al_dif + age", df)
# #outcome, predictors = patsy.dmatrices("behav ~ age * exp_dif", df)
# mod = sm.OLS(outcome, predictors)
# res = mod.fit()

# # Check out the results
# print(res.summary())

In [ ]:
#
al_ = np.mean(all_exps[:, YNG_INDS, :], 0)

exp_st_yng = np.vstack([all_exps[0, YNG_INDS, :], all_exps[1, YNG_INDS, :], all_exps[2, YNG_INDS, :]])
exp_st_old = np.vstack([all_exps[0, OLD_INDS, :], all_exps[1, OLD_INDS, :], all_exps[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

In [ ]:
# Get the change in behavioural scores between highes & lowest loads
delta_behavs = []
temp_df = behav_dat[behav_dat.Age == 1]
for subj_id in set(temp_df.SubjID.values):
    temp = behav_dat[behav_dat.SubjID == subj_id]
    delta_behavs.append(temp[temp.Load == 3]["d'"].values[0] - temp[temp.Load == 1]["d'"].values[0])
delta_behavs = np.array(delta_behavs)

In [ ]:
#behav_dat.Age == 2

In [ ]:
# Get the change in physiological scores between highes & lowest loads
temp_dat = all_exps
temp = temp_dat[2, :, :] - temp_dat[0, :, :]
temp[np.isnan(temp)] = 0
delta_phys = temp[:, 2] - temp[:, 0]

In [ ]:
#print('ALL:', pearsonr(delta_behavs, delta_phys[OLD_INDS]))
#plt.scatter(delta_behavs, delta_phys[OLD_INDS])

In [ ]:
#print('ALL:', pearsonr(delta_behavs, delta_phys[YNG_INDS]))
#plt.scatter(delta_behavs, delta_phys[YNG_INDS])

In [ ]:
# plt.scatter(delta_behavs[OLD_INDS], delta_phys[OLD_INDS], label='OLD')
# print('OLD:', pearsonr(delta_behavs[OLD_INDS], delta_phys[OLD_INDS]))
# plt.scatter(delta_behavs[YNG_INDS], delta_phys[YNG_INDS], label='YNG')
# print('YNG:', pearsonr(delta_behavs[YNG_INDS], delta_phys[YNG_INDS]))
# plt.legend()
# plt.xlabel('Delta Behav');
# plt.ylabel('Delta Physiology');

In [ ]:
#cur_dat = canalpha
#cur_dat = all_exps
cur_dat = all_alphas

# Get phys-responsiveness, as late minus pre
temp = cur_dat[:, :, 2] - cur_dat[:, :, 1]
# Or - just take late
#temp = cur_dat[:, :, 2]

# Replace any nans as zeros
temp[np.isnan(temp)] = 0

# Get delta physiology as high load minus low load
delta_phys = temp[2, :] - temp[0, :]

In [ ]:
# print('ALL:', pearsonr(delta_behavs, delta_phys))
# print('OLD:', pearsonr(delta_behavs[OLD_INDS], delta_phys[OLD_INDS]))
# print('YNG:', pearsonr(delta_behavs[YNG_INDS], delta_phys[YNG_INDS]))

# plt.scatter(delta_behavs[OLD_INDS], delta_phys[OLD_INDS], label='OLD')
# plt.scatter(delta_behavs[YNG_INDS], delta_phys[YNG_INDS], label='YNG')

# #plt.ylim([-0.0000005, 0.0000005])

# plt.legend()
# plt.xlabel('Delta Behav');
# plt.ylabel('Delta Physiology');

In [ ]:
# Check for missing data
for si, dd in zip(range(31), sum(np.isnan(all_alphas), 0).sum(1)):
    print(si, '\t', dd)

# Predict canonalpha from FOOOF alpha + chi

1) Canonical Alpha vs. FOOOF alpha

2) Canonical Alpha vs. Chi

3) Canonical Alpha cs. FOOOF alpha + chi

In [ ]:
from sklearn import preprocessing

In [ ]:
can_scaled = preprocessing.scale(canalpha.flatten())
alf_scaled = preprocessing.scale(all_alphas.flatten())
exp_scaled = preprocessing.scale(all_exps.flatten())

In [ ]:
plt.scatter(alf_scaled, can_scaled)

In [ ]:
plt.scatter(exp_scaled, can_scaled)

In [ ]:
plt.scatter(exp_scaled, alf_scaled)

In [ ]:
plt.scatter(canalpha.flatten(), all_alphas.flatten())
plt.xlim([canalpha.flatten().min(), canalpha.flatten().max()]);

In [ ]:
len(alf_scaled)

In [ ]:
len(exp_scaled)

In [ ]:
# Build the dataframe
df = pd.DataFrame()

df['exp'] = np.concatenate([exp_dif_yng, exp_dif_old])
df['fal'] = np.concatenate([al_dif_yng, al_dif_old])
df['canal'] = np.concatenate([canal_dif_yng, canal_dif_old])
#df['exp'] = all_exps[:, :, 1].flatten()
#df['fal'] = all_alphas[:, :, 1].flatten()
#df['canal'] = canalpha[:, :, 1].flatten()

outcome, predictors = patsy.dmatrices("canal ~ fal * exp", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
print(res.summary())

In [ ]:
# Get the original values and predictions of alpha
canal_vals = df['canal'].values[~np.isnan(df['canal'])]
canalpha_predictions = res.predict(predictors)

In [ ]:
# Check the correlation between measured values and predictions
plt.scatter(canalpha_predictions, canal_vals)
plt.xlim([outcome.min(), outcome.max()]);
plt.ylim([outcome.min(), outcome.max()]);
nan_corr(canal_vals, canalpha_predictions)

In [ ]:
print_stat('YNG:', *nan_corr(al_dif_yng, al_cf_dif_yng))
print_stat('OLD:', *nan_corr(al_dif_old, al_cf_dif_old))
print_stat('ALL:', *nan_corr(np.concatenate([al_dif_yng, al_dif_old]),
                             np.concatenate([al_cf_dif_yng, al_cf_dif_old])))


plt.scatter(np.concatenate([al_dif_yng, al_dif_old]), np.concatenate([al_cf_dif_yng, al_cf_dif_old]))
nan_corr(np.concatenate([al_dif_yng, al_dif_old]), np.concatenate([al_cf_dif_yng, al_cf_dif_old]))